In [1]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:07<00:00, 846kB/s] 


In [101]:
# Run inference on an image
results = model('./Dataset/download/Fruits/31.jpeg' , conf = 0.5)  # results list


# View results

boxex = results[0].boxes.cpu().numpy()
class_labels = boxex.cls

class_labels = set(class_labels)
names = results[0].names

objects = [ names[i] for i in class_labels ]

objects

    
    


image 1/1 /home/epsilon/Desktop/Google-lens-prototype/Dataset/download/Fruits/31.jpeg: 640x512 1 orange, 1 refrigerator, 77.0ms
Speed: 1.7ms preprocess, 77.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 512)


['refrigerator', 'orange']